In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras


print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [4]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

4422102/4422102 [==============================] - 2s 0us/step

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)

In [6]:
model = keras.models.Sequential()
#添加卷积层，filters输出有多少通道，就是有多少卷积核，kernel_size卷积核的大小，
# padding是否加上padding，same代表输出和输入大小一样，1代表通道数目是1
model.add(keras.layers.Conv2D(filters=32, kernel_size=3,
                              padding='same',
                              activation='relu',
                              input_shape=(28, 28, 1)))
model.add(keras.layers.Conv2D(filters=32, kernel_size=3,
                              padding='same',
                              activation='relu'))
#添加池化层，pool_size是窗口大小，步长默认和窗口大小相等
model.add(keras.layers.MaxPool2D(pool_size=2))
#为了缓解损失，所以filters翻倍
model.add(keras.layers.Conv2D(filters=64, kernel_size=3,
                              padding='same',
                              activation='relu'))
model.add(keras.layers.Conv2D(filters=64, kernel_size=3,
                              padding='same',
                              activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=128, kernel_size=3,
                              padding='same',
                              activation='relu'))
model.add(keras.layers.Conv2D(filters=128, kernel_size=3,
                              padding='same',
                              activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer = "sgd",
              metrics = ["accuracy"])

In [7]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                   

In [8]:
model.variables

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32, numpy=
 array([[[[-0.13424508, -0.05933636,  0.00568683, -0.01886322,
           -0.0835022 , -0.0387536 , -0.02725745, -0.07632311,
           -0.06834792, -0.04667236,  0.11829098,  0.00278601,
           -0.0860935 , -0.06942483, -0.04224806, -0.0696409 ,
           -0.03834529,  0.0657406 , -0.07950791,  0.09858753,
           -0.08113267,  0.06931795,  0.01383318,  0.10705055,
           -0.09334123, -0.09491956, -0.13577469,  0.09116071,
            0.1242507 ,  0.0336667 , -0.08405942,  0.09799176]],
 
         [[-0.08385888,  0.04846747,  0.10426976, -0.1260295 ,
           -0.07484409, -0.02521672, -0.11982961,  0.13937692,
            0.00431372,  0.0049036 ,  0.09297548,  0.06705861,
            0.09892599, -0.09695904, -0.06682763, -0.0190882 ,
           -0.07544491, -0.10045683,  0.04131904, -0.07520781,
           -0.10472083,  0.1194783 ,  0.06697379, -0.00041123,
            0.00816672, -0.07369073,  0.

In [9]:
!ls

1.py
2023-7-19.ipynb
2023-7-28缁冧範CNN.ipynb
23-7-18-1 缁冧範.ipynb
23-7-24.ipynb
7.23-exercise.ipynb
722缁冧範.ipynb
cal_housing_py3.pkz
data
exercise.py
generate_csv
main.py
temp.csv
缁冧範2023-7-21.ipynb
鑰佸笀鐨勭粌涔�.ipynb


In [11]:
!rm -rf cnn-relu-callbacks

In [ ]:
logdir = './cnn-relu-callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,
                                 "fashion_mnist_model.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,
                                    save_best_only = True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
]
history = model.fit(x_train_scaled, y_train, epochs=10,
                    validation_data=(x_valid_scaled, y_valid),
                    callbacks = callbacks)

Epoch 1/10
1719/1719 [==============================] - 153s 88ms/step - loss: 0.7581 - accuracy: 0.7237 - val_loss: 0.4672 - val_accuracy: 0.8332

C:\Users\Administrator\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1719/1719 [==============================] - 153s 89ms/step - loss: 0.4049 - accuracy: 0.8507 - val_loss: 0.3494 - val_accuracy: 0.8738

In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'Col-1': [1, 2, 3,4], 'Col-2': [4, 5, 6,7]})
print(df)
print("*"*50)
res1 = df.transform(np.square)#平方
print(res1)
print("*"*50)
res2 = df.transform(np.sqrt)#开平方根
print(res2)
print("*"*50)
res3 = df.transform(lambda x: x*10)#匿名函数,相当于让x->x*10
print(res3)
print("*"*50)


   Col-1  Col-2
0      1      4
1      2      5
2      3      6
3      4      7
**************************************************
   Col-1  Col-2
0      1     16
1      4     25
2      9     36
3     16     49
**************************************************
      Col-1     Col-2
0  1.000000  2.000000
1  1.414214  2.236068
2  1.732051  2.449490
3  2.000000  2.645751
**************************************************
   Col-1  Col-2
0     10     40
1     20     50
2     30     60
3     40     70
**************************************************
